In [1]:
from pyspark.sql import SQLContext

In [2]:
sqlContext = SQLContext(sc)

In [3]:
!rm world_bank.json.gz -f
!wget https://raw.githubusercontent.com/bradenrc/sparksql_pot/master/world_bank.json.gz

--2018-06-04 08:23:34--  https://raw.githubusercontent.com/bradenrc/sparksql_pot/master/world_bank.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 446287 (436K) [application/octet-stream]
Saving to: ‘world_bank.json.gz’

100%[======================================>] 446,287     --.-K/s   in 0.02s   

2018-06-04 08:23:34 (21.1 MB/s) - ‘world_bank.json.gz’ saved [446287/446287]



In [4]:
example1_df = sqlContext.read.json("world_bank.json.gz")

In [5]:
print example1_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- approvalfy: string (nullable = true)
 |-- board_approval_month: string (nullable = true)
 |-- boardapprovaldate: string (nullable = true)
 |-- borrower: string (nullable = true)
 |-- closingdate: string (nullable = true)
 |-- country_namecode: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- countryname: string (nullable = true)
 |-- countryshortname: string (nullable = true)
 |-- docty: string (nullable = true)
 |-- envassesmentcategorycode: string (nullable = true)
 |-- grantamt: long (nullable = true)
 |-- ibrdcommamt: long (nullable = true)
 |-- id: string (nullable = true)
 |-- idacommamt: long (nullable = true)
 |-- impagency: string (nullable = true)
 |-- lendinginstr: string (nullable = true)
 |-- lendinginstrtype: string (nullable = true)
 |-- lendprojectcost: long (nullable = true)
 |-- majorsector_percent: array (nullable = true)
 |    |-- element: struct (containsNu

In [6]:
for row in example1_df.take(2):
    print row
    print "*"*20

Row(_id=Row($oid=u'52b213b38594d8a2be17c780'), approvalfy=u'1999', board_approval_month=u'November', boardapprovaldate=u'2013-11-12T00:00:00Z', borrower=u'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA', closingdate=u'2018-07-07T00:00:00Z', country_namecode=u'Federal Democratic Republic of Ethiopia!$!ET', countrycode=u'ET', countryname=u'Federal Democratic Republic of Ethiopia', countryshortname=u'Ethiopia', docty=u'Project Information Document,Indigenous Peoples Plan,Project Information Document', envassesmentcategorycode=u'C', grantamt=0, ibrdcommamt=0, id=u'P129828', idacommamt=130000000, impagency=u'MINISTRY OF EDUCATION', lendinginstr=u'Investment Project Financing', lendinginstrtype=u'IN', lendprojectcost=550000000, majorsector_percent=[Row(Name=u'Education', Percent=46), Row(Name=u'Education', Percent=26), Row(Name=u'Public Administration, Law, and Justice', Percent=16), Row(Name=u'Education', Percent=12)], mjsector_namecode=[Row(code=u'EX', name=u'Education'), Row(code=u'EX', name=u'E

In [7]:
example1_df.registerTempTable("world_bank")

In [8]:
temp_df = sqlContext.sql("select * from world_bank")

In [9]:
print type(temp_df)

<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
print temp_df

DataFrame[_id: struct<$oid:string>, approvalfy: string, board_approval_month: string, boardapprovaldate: string, borrower: string, closingdate: string, country_namecode: string, countrycode: string, countryname: string, countryshortname: string, docty: string, envassesmentcategorycode: string, grantamt: bigint, ibrdcommamt: bigint, id: string, idacommamt: bigint, impagency: string, lendinginstr: string, lendinginstrtype: string, lendprojectcost: bigint, majorsector_percent: array<struct<Name:string,Percent:bigint>>, mjsector_namecode: array<struct<code:string,name:string>>, mjtheme: array<string>, mjtheme_namecode: array<struct<code:string,name:string>>, mjthemecode: string, prodline: string, prodlinetext: string, productlinetype: string, project_abstract: struct<cdata:string>, project_name: string, projectdocs: array<struct<DocDate:string,DocType:string,DocTypeDesc:string,DocURL:string,EntityID:string>>, projectfinancialtype: string, projectstatusdisplay: string, regionname: string, s

In [11]:
import pandas as pd

In [12]:
sqlContext.sql("select * from world_bank limit 2").toPandas()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,"(52b213b38594d8a2be17c780,)",1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"(Education for all, 100)","[(65, Education for all)]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,"(52b213b38594d8a2be17c781,)",2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,None,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"(Other economic management, 30)","[(24, Other economic management), (54, Social ...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [13]:
query = """
select
    regionname ,
    count(*) as project_count
from world_bank
group by regionname 
order by count(*) desc
"""

In [14]:
sqlContext.sql(query).toPandas()

,regionname,project_count
0,Africa,152
1,East Asia and Pacific,100
2,Europe and Central Asia,74
3,South Asia,65
4,Middle East and North Africa,54
5,Latin America and Caribbean,53
6,Other,2


In [15]:
query = """

select * from
    (select
        regionname ,
        count(*) as project_count
    from world_bank
    group by regionname 
    order by count(*) desc) table_alias
limit 2
"""
sqlContext.sql(query).toPandas()

,regionname,project_count
0,Africa,152
1,East Asia and Pacific,100


In [16]:
example1_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- approvalfy: string (nullable = true)
 |-- board_approval_month: string (nullable = true)
 |-- boardapprovaldate: string (nullable = true)
 |-- borrower: string (nullable = true)
 |-- closingdate: string (nullable = true)
 |-- country_namecode: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- countryname: string (nullable = true)
 |-- countryshortname: string (nullable = true)
 |-- docty: string (nullable = true)
 |-- envassesmentcategorycode: string (nullable = true)
 |-- grantamt: long (nullable = true)
 |-- ibrdcommamt: long (nullable = true)
 |-- id: string (nullable = true)
 |-- idacommamt: long (nullable = true)
 |-- impagency: string (nullable = true)
 |-- lendinginstr: string (nullable = true)
 |-- lendinginstrtype: string (nullable = true)
 |-- lendprojectcost: long (nullable = true)
 |-- majorsector_percent: array (nullable = true)
 |    |-- element: struct (containsNu

In [17]:
sql = "select sector1.Name from world_bank limit 2"
sqlContext.sql(sql).show()

+--------------------+
|                Name|
+--------------------+
|   Primary education|
|Public administra...|
+--------------------+



In [18]:
import random
data_e2 = []
for x in range(1,6):
    random_int = int(random.random()*10)
    data_e2.append([x, random_int, random_int^2])

In [19]:
data_e2

[[1, 4, 6], [2, 9, 11], [3, 0, 2], [4, 9, 11], [5, 6, 4]]

In [20]:
rdd_example2 = sc.parallelize(data_e2)

In [21]:
rdd_example2.collect()

[[1, 4, 6], [2, 9, 11], [3, 0, 2], [4, 9, 11], [5, 6, 4]]

In [22]:
from pyspark.sql.types import *

schemaString = "ID VAL1 VAL2"

In [23]:
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]

In [24]:
schema = StructType(fields)

In [25]:
schema

StructType(List(StructField(ID,StringType,true),StructField(VAL1,StringType,true),StructField(VAL2,StringType,true)))

In [26]:
schemaExample = sqlContext.createDataFrame(rdd_example2, schema)

In [28]:
schemaExample.registerTempTable("example2")

In [29]:
print schemaExample.collect()

[Row(ID=u'1', VAL1=u'4', VAL2=u'6'), Row(ID=u'2', VAL1=u'9', VAL2=u'11'), Row(ID=u'3', VAL1=u'0', VAL2=u'2'), Row(ID=u'4', VAL1=u'9', VAL2=u'11'), Row(ID=u'5', VAL1=u'6', VAL2=u'4')]


In [31]:
for row in schemaExample.take(2):
    print row.ID, row.VAL1, row.VAL2

1 4 6
2 9 11


In [32]:
sqlContext.sql("select * from example2").toPandas()

,ID,VAL1,VAL2
0,1,4,6
1,2,9,11
2,3,0,2
3,4,9,11
4,5,6,4


In [35]:
from pyspark.sql import Row
rdd_example3 = rdd_example2.map(lambda x: Row(id = x[0], val1 = x[1], val2 = x[2]))

In [36]:
print rdd_example3.collect()

[Row(id=1, val1=4, val2=6), Row(id=2, val1=9, val2=11), Row(id=3, val1=0, val2=2), Row(id=4, val1=9, val2=11), Row(id=5, val1=6, val2=4)]


In [37]:
df_example3 = rdd_example3.toDF()

In [38]:
df_example3.registerTempTable("example3")

In [40]:
sqlContext.sql("select * from example3").toPandas()

,id,val1,val2
0,1,4,6
1,2,9,11
2,3,0,2
3,4,9,11
4,5,6,4


In [41]:
query = """
select
    *
from
    example2 e2
inner join example3 e3 on
    e2.ID = e3.id
"""

print sqlContext.sql(query).toPandas()

  ID VAL1 VAL2  id  val1  val2
0  1    4    6   1     4     6
1  4    9   11   4     9    11
2  3    0    2   3     0     2
3  2    9   11   2     9    11
4  5    6    4   5     6     4


In [42]:
df_example4 = df_example3.join(schemaExample, schemaExample["ID"] == df_example3["id"])

In [43]:
for row in df_example4.take(5):
    print row

Row(id=1, val1=4, val2=6, ID=u'1', VAL1=u'4', VAL2=u'6')
Row(id=4, val1=9, val2=11, ID=u'4', VAL1=u'9', VAL2=u'11')
Row(id=3, val1=0, val2=2, ID=u'3', VAL1=u'0', VAL2=u'2')
Row(id=2, val1=9, val2=11, ID=u'2', VAL1=u'9', VAL2=u'11')
Row(id=5, val1=6, val2=4, ID=u'5', VAL1=u'6', VAL2=u'4')


In [44]:
def simple_function(v):
    return int(v*10)
print simple_function(3)

30


In [46]:
sqlContext.registerFunction("simple_function", simple_function)

In [47]:
query = """
select ID,
VAL1,
VAL2,
simple_function(VAL1) as s_VAL1,
simple_function(VAL2) as s_VAL2

from example2
"""

sqlContext.sql(query).toPandas()

,ID,VAL1,VAL2,s_VAL1,s_VAL2
0,1,4,6,4444444444,6666666666
1,2,9,11,9999999999,11111111111111111111
2,3,0,2,0,2222222222
3,4,9,11,9999999999,11111111111111111111
4,5,6,4,6666666666,4444444444


In [48]:
query = """
select ID,
VAL1,
VAL2,
simple_function(cast(VAL1 as int)) as s_VAL1,
simple_function(cast(VAL2 as int)) as s_VAL2

from example2
"""

sqlContext.sql(query).toPandas()

,ID,VAL1,VAL2,s_VAL1,s_VAL2
0,1,4,6,40,60
1,2,9,11,90,110
2,3,0,2,0,20
3,4,9,11,90,110
4,5,6,4,60,40


In [51]:
!rm SIGHTINGS.csv -f
!wget https://www.quandl.com/api/v3/datasets/NUFORC/SIGHTINGS.csv

--2018-06-04 08:46:07--  https://www.quandl.com/api/v3/datasets/NUFORC/SIGHTINGS.csv
Resolving www.quandl.com (www.quandl.com)... 104.25.53.103, 104.25.54.103, 2400:cb00:2048:1::6819:3667, ...
Connecting to www.quandl.com (www.quandl.com)|104.25.53.103|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2018-06-04 08:46:08 ERROR 404: Not Found.



In [50]:
pandas_df = pd.read_csv("./SIGHTINGS.csv")
pandas.head()

IOError: File ./SIGHTINGS.csv does not exist